<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/modelToDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##モデルをDB化

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import IPython
#bz2ファイル解凍
#!bunzip2 -k -q "./drive/My Drive/NLP/20170201.tar.bz2"

#tarファイル解凍
!tar -xvf "./drive/My Drive/NLP/20170201.tar"

entity_vector/
entity_vector/entity_vector.model.txt
entity_vector/entity_vector.model.bin


In [24]:
from gensim.models import KeyedVectors

#解凍時のみ
#model_dir = 'entity_vector/entity_vector.model.bin'
#model = KeyedVectors.load_word2vec_format(model_dir, binary=True,limit=500000)

#解凍後確認
results = model.most_similar(positive=[u'リンゴ','逆'])
for result in results:
  print(result)

('ナッツ', 0.608566403388977)
('バナナ', 0.6054157018661499)
('蜜', 0.6049777865409851)
('トマト', 0.6029523015022278)
('木の実', 0.6004878878593445)
('ハエ', 0.5923978686332703)
('ミルク', 0.5920907855033875)
('ニンジン', 0.5913463830947876)
('タネ', 0.5892855525016785)
('キャンディー', 0.5888605117797852)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [25]:
!pip install mecab-python3

     |████████████████████████████████| 15.9MB 3.5MB/s 


##語彙-対義語候補 対応CSV作成

In [26]:
import re
import pandas as pd

cnt = 1
model_vocab_list = []

for words, v in model.vocab.items():

  words = words.replace('_', '')
  
  #固有名詞用の括弧を削除
  words = words.replace('[', '').replace(']', '')

  #-----------------------------------------------------  

  # 半角記号+半角数字
  #p = re.compile("[!-@[-`{-~]")
  # 半角記号+半角数字+半角英字
  #((ほぼ)日本語のみのDB作成用)
  p = re.compile("[!-~]")

  # 先頭文字
  top_char = words[0:1]

  #-----------------------------------------------------  

  #補足情報の_を削除
  words = words.replace("_","")
  #括弧文字を抽出
  regex = re.compile(".*?\((.*?)\)")
  #括弧文字をlist型で返却
  ret_list = re.findall(regex, words)

  #-----------------------------------------------------  

  #除外文字なし
  if p.match(top_char) is None:
    #括弧文字がある場合は無効化
    if len(ret_list) > 0: 
      words = words.replace("(" + ret_list[0] + ")",'')
      model_vocab_list.append(words)
    else:
      model_vocab_list.append(words)
      
  if cnt % 50000 == 0:
    print(cnt,"件 完了");
    
  cnt = cnt + 1

#重複削除
model_vocab_uni_list= list(set(model_vocab_list))
  
#保存
df = pd.DataFrame(model_vocab_uni_list)
df.to_csv('./drive/My Drive/NLP/model_vocab_list.csv', header=False, index=False)

50000 件 完了
100000 件 完了
150000 件 完了
200000 件 完了
250000 件 完了
300000 件 完了
350000 件 完了
400000 件 完了
450000 件 完了
500000 件 完了


In [63]:
import pandas as pd

drive.mount('/content/drive')
csv_dir = './drive/My Drive/NLP/model_vocab_list.csv'
df = pd.read_csv(csv_dir,names=['words'])
df = df.sort_values('words', ascending=True)

print(df.count())
print(df[df["words"]=="ゆず"])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
words    358056
dtype: int64
       words
312144    ゆず


In [0]:
import MeCab
#括弧記号無効化
import lib_delete_brackets

def search_similar_texts(p_words):
  pass

def wordRevChange(words,gyaku,inherent_words,model,tokenizer):

  word_cng_list = []

  #**************************************************************************
  #そのまま対義化
  #**************************************************************************
  asitis_cng_words1 = ''
  asitis_cng_words2 = ''
  try:

    rvs_wd_list = model.most_similar(positive=[inherent_words,gyaku])
    inherent_words_bk = inherent_words.replace('[',"").replace(']',"")
    
    index = 0
    rvs_wd_rt_list = []

    for index in range(len(rvs_wd_list)):
      if len(rvs_wd_list[index][0]) < 10 and inherent_words_bk != str(rvs_wd_list[index][0]):

        #対義語候補格納
        rvs_wd_rt_list.append(rvs_wd_list[index][0])

    asitis_cng_words1 = rvs_wd_rt_list[0]
    asitis_cng_words2 = rvs_wd_rt_list[1]
      
  except KeyError as error:
    #辞書に登録の無い単語の場合
    #search_similar_texts(inherent_words)
    pass
  except IndexError as error:
    #辞書に登録の無い単語の場合
    #search_similar_texts(inherent_words)
    pass

  #**************************************************************************
  #形態素分析後に対義化
  #**************************************************************************
  node = tokenizer.parseToNode(words)

  while node:
    cut_wd = node.surface

    if node.feature.split(",")[0] == u"名詞":
      try:
        rvs_wd_list = model.most_similar(positive=[cut_wd,gyaku])
        for i in range(2):
          rvs_wd = rvs_wd_list[i][0]
          if cut_wd != rvs_wd.replace('[',"").replace(']',""):
            #◆結合
            word_cng_list.append(rvs_wd.replace('[',"").replace(']',""))
            break
      except KeyError as error:
        #辞書に登録の無い単語の場合
        #search_similar_texts(cut_wd)
        pass
    elif (node.feature.split(",")[0] == u"動詞" or
      node.feature.split(",")[0] == u"形容詞" or
      node.feature.split(",")[0] == u"副詞" or
      node.feature.split(",")[0] == u"感動詞"):
      cut_wd = node.feature.split(",")[6]

      try:
        rvs_wd_list = model.most_similar(positive=[cut_wd,gyaku])
        for i in range(2):
          rvs_wd = rvs_wd_list[i][0]
          if cut_wd != rvs_wd.replace('[',"").replace(']',""):
            #◆結合
            word_cng_list.append(rvs_wd.replace('[',"").replace(']',""))
            break
      except KeyError as error:
        #辞書に登録の無い単語の場合
        #search_similar_texts(cut_wd)
        pass

    else:
      #◆結合
      word_cng_list.append(cut_wd.replace('[',"").replace(']',""))

    node = node.next

  asitis_cng_words1 = asitis_cng_words1.replace('[',"").replace(']',"").replace('_',"")
  asitis_cng_words2 = asitis_cng_words2.replace('[',"").replace(']',"").replace('_',"")

  result_list = []

  result_list.append(words)
  result_list.append(lib_delete_brackets.delete_brackets(asitis_cng_words1))
  result_list.append(lib_delete_brackets.delete_brackets(asitis_cng_words2))
  result_list.append(''.join(word_cng_list))

  return result_list


In [0]:
import re
import pandas as pd
import MeCab
#対義語生成
#import lib_wordRevChange as lw

tokenizer = MeCab.Tagger("-Ochasen")

cnt = 1
save_list = []
jpn_start_flg = 0

#列[words]をnumpyに変換
val = df.words.values

for idx in range(df.shape[0]):

  words = str(val[idx])
  
  #日本語単語の開始判定
  if words[0:1] == "あ":
    jpn_start_flg = 1
  
  if jpn_start_flg == 1:
    if len(words) <= 10:
      if words.isdecimal() is False:
        if '・' not in words:
          gyaku = u"逆"
          inherent_words = '[' + words + ']'  
          rev_list = wordRevChange(words,gyaku,inherent_words,model,tokenizer)
          save_list.append(rev_list)

  if cnt % 1000 == 0:
    print(cnt,"件 終了")

    if cnt % 3000 == 0:
      #Gドライブ再マウント
      drive.mount('/content/drive')  
      #保存
      df2 = pd.DataFrame(save_list)
      df2.to_csv('./drive/My Drive/NLP/modelToCsv.csv', header=False, index=False)
    
  cnt = cnt + 1
  
#Gドライブ再マウント
drive.mount('/content/drive')  
#保存
df2 = pd.DataFrame(save_list)
df2.to_csv('./drive/My Drive/NLP/modelToCsv.csv', header=False, index=False)


##以下、検証用

In [118]:

rt = model.most_similar(positive=["歩く","逆"])
rt

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('曲がる', 0.670616865158081),
 ('逃げる', 0.6605823040008545),
 ('寄る', 0.6599925756454468),
 ('泳ぐ', 0.6526366472244263),
 ('滑る', 0.6492980718612671),
 ('行く', 0.6289190053939819),
 ('転がる', 0.6097988486289978),
 ('持ち上げる', 0.6081393957138062),
 ('曲げる', 0.6051719188690186),
 ('触る', 0.6028169393539429)]

In [50]:
tokenizer = MeCab.Tagger("-Ochasen")
node = tokenizer.parseToNode("さようなら")

while node:
  print(node.feature.split(",")[0])
  node = node.next

BOS/EOS
感動詞
BOS/EOS


##以下、検証用2

In [7]:
!pip install mecab-python3

     |████████████████████████████████| 15.9MB 2.7MB/s 


In [5]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
csv_dir = './drive/My Drive/NLP/modelToCsv_500000.csv'
df_ant = pd.read_csv(csv_dir,names=['words','ant1','ant2','ant3'])
df_ant = df_ant.sort_values('words', ascending=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import MeCab
import numpy as np

def get_ant_word(words):

  word_cng_list = []

  #-------------------------------------------------
  # そのまま対義語化
  #-------------------------------------------------
  try:
    rvs_wd = df_ant[df_ant["words"] == words].values[0][1]
    if rvs_wd == np.nan:
      rvs_wd = df_ant[df_ant["words"] == words].values[0][2]
      if  rvs_wd == np.nan:
        rvs_wd = df_ant[df_ant["words"] == words].values[0][3]

    word_cng_list.append(rvs_wd)

  except IndexError as error:
    #辞書に登録の無い単語の場合
    pass

  #-------------------------------------------------
  # 形態素分析後に対義語化
  #-------------------------------------------------
  tokenizer = MeCab.Tagger("-Ochasen")
  node = tokenizer.parseToNode(words)
  ant_word = ''

  while node:
    #分かち書きの単語を取得
    cut_wd = node.surface

    if cut_wd != np.nan and cut_wd != '':
      if node.feature.split(",")[0] == u"名詞":
        try:
          rvs_wd = df_ant[df_ant["words"] == cut_wd].values[0][1]
          if rvs_wd is np.nan:
            rvs_wd = df_ant[df_ant["words"] == cut_wd].values[0][2]
            if  rvs_wd is np.nan:
              rvs_wd = df_ant[df_ant["words"] == cut_wd].values[0][3]

          ant_word = ant_word + rvs_wd

        except IndexError as error:
          #辞書に登録の無い単語の場合
          pass

      elif (node.feature.split(",")[0] == u"動詞" or
        node.feature.split(",")[0] == u"形容詞" or
        node.feature.split(",")[0] == u"副詞" or
        node.feature.split(",")[0] == u"感動詞"):

        #分かち書きの単語を取得
        cut_wd = node.feature.split(",")[6]

        try:
          rvs_wd = df_ant[df_ant["words"] == cut_wd].values[0][1]
          if rvs_wd is np.nan:
            rvs_wd = df_ant[df_ant["words"] == cut_wd].values[0][2]
            if rvs_wd is np.nan:
              rvs_wd = df_ant[df_ant["words"] == cut_wd].values[0][3]

          ant_word = ant_word + rvs_wd

        except IndexError as error:
          #辞書に登録の無い単語の場合
          pass

      else:
        #◆結合
        ant_word = ant_word + cut_wd

    node = node.next

  word_cng_list.append(ant_word)

  word_cng_list_uni = list(set(word_cng_list))
  
  return word_cng_list_uni

In [113]:
print(get_ant_word("波乗りジョニー"))

['ハイジャンプサム', 'MISS']


In [116]:
df_ant[df_ant["words"] == "波乗りジョニー"]

,words,ant1,ant2,ant3
233565,波乗りジョニー,MISS,与作,ハイジャンプサム
